<!--NAVIGATION-->
< [Aggregation and Grouping](03.08-Aggregation-and-Grouping.ipynb) | [Contents](Index.ipynb) | [Vectorized String Operations](03.10-Working-With-Strings.ipynb) >

# 3.10 数据透视表

我们已经介绍过``GroupBy``抽象类是如何探索数据集内部的关联性的了。**数据透视表**（pivot table）是一种类似的操作方法，常见于Excel与类似的表格应用中。数据透视表将每一列数据作为输入，输出将数据不断细分成多个维度累计信息的二维数据表。

人们有时容易弄混数据透视表与``GroupBy``，但我觉得数据透视表更像是一种**多维**的``GroupBy``累计操作。也就是说，虽然你也可以分割-应用-组合，但是分割与组合不是发生在一维索引上，而是在二维网格上（行列同时分组）。

## 1. 演示数据透视表

这一节的示例将采用*泰坦尼克号*的乘客信息数据库来演示，可以在Seaborn程序库获取,代码如下所示：（如果获取不了，没关系，我已经下载好了，请运行
```
titanic=pd.read_csv('data/titanic.csv',index_col='number')
```

In [17]:
#import numpy as np
#import pandas as pd
#import seaborn as sns
#titanic = sns.load_dataset('titanic')
#titanic.to_csv('data/titanic.csv')
titanic=pd.read_csv('data/titanic.csv',index_col='number')

In [18]:
titanic.head()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
number,,,,,,,,,,,,,,,
0,0,3,male,22.0,1,0,7.2500,S,Third,man,True,NaN,Southampton,no,False
1,1,1,female,38.0,1,0,71.2833,C,First,woman,False,C,Cherbourg,yes,False
2,1,3,female,26.0,0,0,7.9250,S,Third,woman,False,NaN,Southampton,yes,True
3,1,1,female,35.0,1,0,53.1000,S,First,woman,False,C,Southampton,yes,False
4,0,3,male,35.0,0,0,8.0500,S,Third,man,True,NaN,Southampton,no,True


这份数据包含了泰坦尼克号的每位乘客的大量信息，包括性别（gender）、年龄（age）、船舱等级（class）和船票价格（fare paid）等。

## 2. 手工制作数据透视表

在研究这些数据之前，先将它们按照性别、最终生还状态或其他组合属性进行分组。如果你看过前面的章节，你可能会用``GroupBy``来实现，例如这样统计不同性别乘客的生还率：

In [19]:
titanic.groupby('sex')[['survived']].mean()

,survived
sex,
female,0.742038
male,0.188908


这组数据会立刻给我们一个直观感受：总体来说，有四分之三的女性被救，但只有五分之一的男性被救！

这组数据很有用，但是我们可能还想进一步探索，同时观察不同性别与船舱等级的生还情况。根据``GroupBy``的操作流程，我们也许能够实现想要的结果：将船舱等级（``'class'``）与性别（``'sex'``）**分组**，然后**选择**生还状态（``'survived'``）列，**应用**均值（``'mean'``）累计函数，再将各组结果**组合**，最后通过**行索引转列索引**操作将最里层的行索引转换成列索引，形成二维数组。代码如下所示：

In [20]:
titanic.groupby(['sex', 'class'])['survived'].aggregate('mean').unstack()

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


虽然这样就可以更清晰地观察乘客性别、船舱等级对其是否生还的影响，但是代码看上去有点复杂。尽管这个管道命令的每一步都是前面介绍过的，但是要理解这个长长的语句可不是那么容易的事。

由于二维的``GroupBy``应用场景非常普遍，因此Pandas提供了一个快捷方式``pivot_table``来快速解决多维的累计分析任务。

## 3. 数据透视表语法

用``DataFrame``的``pivot_table``实现的效果等同于上一节的管道命令的代码：

In [21]:
titanic.pivot_table('survived', index='sex', columns='class')

class,First,Second,Third
sex,,,
female,0.968085,0.921053,0.500000
male,0.368852,0.157407,0.135447


与``groupby``方法相比，这行代码可读性更强，而且取得的结果也一样。可能与你对20世纪初的那场灾难的猜想一致，生还率最高的是船舱等级高的女性。一等舱的女性乘客基本全部生还（hi, Rose!），而三等舱男性乘客的生还率仅为十分之一（sorry, Jack!）。

### 1. 多级数据透视表

与``GroupBy``类似，数据透视表中的分组也可以通过各种参数指定多个等级。例如，我们可能想把年龄（``'age'``）也加进去作为第三个维度，这就可以通过``pd.cut``函数将年龄进行分段：

In [22]:
age = pd.cut(titanic['age'], [0, 18, 80])
titanic.pivot_table('survived', ['sex', age], 'class')

class               First    Second     Third
sex    age                                   
female (0, 18]   0.909091  1.000000  0.511628
       (18, 80]  0.972973  0.900000  0.423729
male   (0, 18]   0.800000  0.600000  0.215686
       (18, 80]  0.375000  0.071429  0.133663

对某一列也可以使用同样的策略——让我们用``pd.qcut``将船票价格按照计数项等分为两份，加入数据透视表看看：

In [23]:
fare = pd.qcut(titanic['fare'], 2)
titanic.pivot_table('survived', ['sex', age], [fare, 'class'])

fare            (-0.001, 14.454]                     (14.454, 512.329]  \
class                      First    Second     Third             First   
sex    age                                                               
female (0, 18]               NaN  1.000000  0.714286          0.909091   
       (18, 80]              NaN  0.880000  0.444444          0.972973   
male   (0, 18]               NaN  0.000000  0.260870          0.800000   
       (18, 80]              0.0  0.098039  0.125000          0.391304   

fare                                 
class              Second     Third  
sex    age                           
female (0, 18]   1.000000  0.318182  
       (18, 80]  0.914286  0.391304  
male   (0, 18]   0.818182  0.178571  
       (18, 80]  0.030303  0.192308

结果是一个带层级索引的四维累计数据表，通过网格显示不同数值之间的相关性。

### 2. 其他数据透视表选项

``DataFrame``的``pivot_table``方法的完整签名如下所示：

```python
# Pandas0.18版的函数签名
DataFrame.pivot_table(data, values=None, index=None, columns=None,
                      aggfunc='mean', fill_value=None, margins=False,
                      dropna=True, margins_name='All')
```
我们已经介绍过前面三个参数了，现在来看看其他参数。``fill_value``和``dropna``这两个参数用于处理缺失值，用法很简单，我们将在后面的示例中演示其用法。

``aggfunc``参数用于设置累计函数类型，默认值是均值（``mean``）。与``GroupBy``的用法一样，累计函数可以用一些常见的字符串（``'sum'``、``'mean'``、``'count'``、``'min'``、``'max'``等）表示，也可以用标准的累计函数（``np.sum()``、``min()``、``sum()``等）表示。另外，还可以通过字典为不同的列指定不同的累计函数：

In [24]:
titanic.pivot_table(index='sex', columns='class',
                    aggfunc={'survived':sum, 'fare':'mean'})

fare                       survived             
class        First     Second      Third    First Second Third
sex                                                           
female  106.125798  21.970121  16.118810       91     70    72
male     67.226127  19.741782  12.661633       45     17    47

需要注意的是，这里忽略了一个参数``values``。当我们为``aggfunc``指定映射关系的时候，待透视的数值就已经确定了。

当需要计算每一组的总数时，可以通过``margins``参数来设置：

In [25]:
titanic.pivot_table('survived', index='sex', columns='class', margins=True)

class,First,Second,Third,All
sex,,,,
female,0.968085,0.921053,0.500000,0.742038
male,0.368852,0.157407,0.135447,0.188908
All,0.629630,0.472826,0.242363,0.383838


这样就可以自动获取不同性别下船舱等级与生还率的相关信息、不同船舱等级下性别与生还率的相关信息，以及全部乘客的生还率为38%。``margin``的标签可以通过``margins_name``参数进行自定义，默认值是``"All"``。

<!--NAVIGATION-->
< [Aggregation and Grouping](03.08-Aggregation-and-Grouping.ipynb) | [Contents](Index.ipynb) | [Vectorized String Operations](03.10-Working-With-Strings.ipynb) >